### this notebook saves the metrics (static ones; and lss ones) for v1.6 dbs.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
import os
import pandas as pd
%matplotlib inline

In [2]:
data_dir = '/global/cscratch1/sd/awan/lsst_output/post_wp_output_v1.6_-0.1cuts/summary_data/'
outdir = '/global/homes/a/awan/LSST/lsstRepos/ObsStrat/postwp/paper-data/summary_csv_v1.6_-0.1cuts/'
os.makedirs(outdir, exist_ok=True)

#### read in the data for the static metrics and store as a dictionary

In [3]:
# read in the data 
data = {}
yr_label = {}
for yr in [1, 3, 6, 10]:
    files = [f for f in os.listdir( data_dir ) if f.endswith('csv') and f.__contains__('y%s_' % yr) and f.startswith('eg_') ]
    for file in files:
        print( file )
        key = 'yr%s_%s' % (yr, file.split('_')[4])
        data[ key ] = pd.read_csv('%s/%s' % (data_dir, file))
        yr_label[ key ] =  r'Y%s (i$>$%s) ' % (yr, file.split('_')[4].split('limi')[-1])
        
print( yr_label )
print(data.keys())
print(data[list(data.keys())[0]].keys() )

eg_footprint_stats_y1_limi24.65_nodither_nside256.csv
eg_footprint_stats_y3_limi25.25_nodither_nside256.csv
eg_footprint_stats_y6_limi25.619999999999997_nodither_nside256.csv
eg_footprint_stats_y10_limi25.9_nodither_nside256.csv
{'yr1_limi24.65': 'Y1 (i$>$24.65) ', 'yr3_limi25.25': 'Y3 (i$>$25.25) ', 'yr6_limi25.619999999999997': 'Y6 (i$>$25.619999999999997) ', 'yr10_limi25.9': 'Y10 (i$>$25.9) '}
dict_keys(['yr1_limi24.65', 'yr3_limi25.25', 'yr6_limi25.619999999999997', 'yr10_limi25.9'])
Index(['dbname', 'Area (deg2)', '$i$-band depth: median',
       ' $i$-band depth: std'],
      dtype='object')


In [4]:
# read in the static metrics
for key in data:
    yr_tag = key.split('_')[0].split('yr')[-1]
    lim_tag = key.split('_')[1].split('limi')[-1]
    print(yr_tag, lim_tag)

    data[key] = data[key].rename(columns={'Area (deg2)': 'Y%s effective survey area ' % (yr_tag),
                                          '$i$-band depth: median': 'median Y%s $i$-band coadded depth in effective survey area' % (yr_tag),
                                          ' $i$-band depth: std': 'std in Y%s $i$-band coadded depth in effective survey area' % (yr_tag)} )
    #fname = 'lss_metrics_%s.csv' % key
    #data[key].to_csv('%s/%s' % (outdir, fname), index=False)

1 24.65
3 25.25
6 25.619999999999997
10 25.9


#### get the order of the dbs

In [5]:
df_in_order = pd.read_csv('%s/given_order_v1.6_2.csv' % outdir, )
df_in_order

,name
0,alt_base_v1.6_10yrs.db
1,barebones_nexp2_v1.6_10yrs.db
2,barebones_v1.6_10yrs.db
3,baseline_nexp1_v1.6_10yrs.db
4,baseline_nexp2_scaleddown_v1.6_10yrs.db
5,baseline_nexp2_v1.6_10yrs.db
6,combo_dust_nexp2_v1.6_10yrs.db
7,combo_dust_v1.6_10yrs.db
8,ddf_heavy_nexp2_v1.6_10yrs.db
9,ddf_heavy_v1.6_10yrs.db


In [6]:
# check some things:
len( list( df_in_order['name'] ) ), len( np.unique( df_in_order['name'] ) )

(32, 32)

In [7]:
yr_key = [f for f in data.keys() if f.__contains__('yr1_')][0]
yr_key

'yr1_limi24.65'

#### find the indices to get the order

In [8]:
# find the indices to get the order
ind = []
for db in df_in_order['name']:
    ind_temp = np.where( data[yr_key]['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
        
    if ind_temp in ind:
        print('%s ind already in. %s' % (ind_temp, db))
    ind +=  list( ind_temp )
    
ind = np.array(ind)
print(ind)

[30 29 23 18 24 12 22  9  4 19  2 11 21 13 26 16  0 28  5 17 14  3  1 20
 15 27  7 25 10 31  8  6]


In [9]:
# check we have the right indices
print( data[yr_key]['dbname'].values[ind[0]],  df_in_order['name'][0] )

# check we have the right number of inds
print( len(ind), len(np.unique(ind)) )

alt_base_v1.6_10yrs alt_base_v1.6_10yrs.db
32 32


#### save the data in the order in which to save things

In [10]:
data.keys()

dict_keys(['yr1_limi24.65', 'yr3_limi25.25', 'yr6_limi25.619999999999997', 'yr10_limi25.9'])

In [11]:
for yr_lim_tag in data:
    for colname in data[yr_lim_tag]:
        if colname != 'dbname':
             df_in_order[colname] = data[yr_lim_tag][colname].values[ind]

In [12]:
df_in_order

,name,Y1 effective survey area,median Y1 $i$-band coadded depth in effective survey area,std in Y1 $i$-band coadded depth in effective survey area,Y3 effective survey area,median Y3 $i$-band coadded depth in effective survey area,std in Y3 $i$-band coadded depth in effective survey area,Y6 effective survey area,median Y6 $i$-band coadded depth in effective survey area,std in Y6 $i$-band coadded depth in effective survey area,Y10 effective survey area,median Y10 $i$-band coadded depth in effective survey area,std in Y10 $i$-band coadded depth in effective survey area
0,alt_base_v1.6_10yrs.db,15325.03,25.09,0.16,15393.01,25.71,0.14,15328.86,26.05,0.13,15346.90,26.26,0.11
1,barebones_nexp2_v1.6_10yrs.db,15163.15,25.20,0.19,15326.81,25.81,0.15,15288.94,26.14,0.14,15282.23,26.40,0.13
2,barebones_v1.6_10yrs.db,15198.09,25.21,0.19,15382.68,25.86,0.15,15355.87,26.19,0.14,15339.35,26.46,0.13
3,baseline_nexp1_v1.6_10yrs.db,15165.04,25.11,0.18,15390.34,25.73,0.13,15353.99,26.07,0.13,15334.00,26.34,0.12
4,baseline_nexp2_scaleddown_v1.6_10yrs.db,14863.26,25.06,0.17,15257.10,25.69,0.13,15177.94,26.03,0.13,15178.68,26.30,0.12
5,baseline_nexp2_v1.6_10yrs.db,14836.77,25.03,0.17,15246.82,25.67,0.13,15188.64,26.01,0.12,15159.64,26.27,0.12
6,combo_dust_nexp2_v1.6_10yrs.db,15765.87,24.97,0.18,17556.45,25.56,0.13,17194.56,25.91,0.13,17087.49,26.18,0.12
7,combo_dust_v1.6_10yrs.db,17161.14,25.03,0.19,18024.20,25.63,0.14,17771.62,25.98,0.14,17693.36,26.24,0.13
8,ddf_heavy_nexp2_v1.6_10yrs.db,14262.06,24.98,0.14,15029.02,25.56,0.10,14827.70,25.89,0.10,14765.80,26.16,0.09
9,ddf_heavy_v1.6_10yrs.db,14944.15,25.06,0.16,15235.75,25.64,0.12,15129.58,25.97,0.11,15089.35,26.23,0.10


In [13]:
cols = df_in_order.columns.tolist()
cols

['name',
 'Y1 effective survey area ',
 'median Y1 $i$-band coadded depth in effective survey area',
 'std in Y1 $i$-band coadded depth in effective survey area',
 'Y3 effective survey area ',
 'median Y3 $i$-band coadded depth in effective survey area',
 'std in Y3 $i$-band coadded depth in effective survey area',
 'Y6 effective survey area ',
 'median Y6 $i$-band coadded depth in effective survey area',
 'std in Y6 $i$-band coadded depth in effective survey area',
 'Y10 effective survey area ',
 'median Y10 $i$-band coadded depth in effective survey area',
 'std in Y10 $i$-band coadded depth in effective survey area']

#### realized that there's too much finagling when sharing data so its best to reorder columns to make sure the order of the columns is the same.

In [14]:
new_cols = ['name',
            'median Y1 $i$-band coadded depth in effective survey area',
            'median Y3 $i$-band coadded depth in effective survey area',
            'median Y6 $i$-band coadded depth in effective survey area',
            'median Y10 $i$-band coadded depth in effective survey area',
            'std in Y1 $i$-band coadded depth in effective survey area',
            'std in Y3 $i$-band coadded depth in effective survey area',
            'std in Y6 $i$-band coadded depth in effective survey area',
            'std in Y10 $i$-band coadded depth in effective survey area',
            'Y1 effective survey area ',
            'Y3 effective survey area ',
            'Y6 effective survey area ',
            'Y10 effective survey area ',]
new_cols

['name',
 'median Y1 $i$-band coadded depth in effective survey area',
 'median Y3 $i$-band coadded depth in effective survey area',
 'median Y6 $i$-band coadded depth in effective survey area',
 'median Y10 $i$-band coadded depth in effective survey area',
 'std in Y1 $i$-band coadded depth in effective survey area',
 'std in Y3 $i$-band coadded depth in effective survey area',
 'std in Y6 $i$-band coadded depth in effective survey area',
 'std in Y10 $i$-band coadded depth in effective survey area',
 'Y1 effective survey area ',
 'Y3 effective survey area ',
 'Y6 effective survey area ',
 'Y10 effective survey area ']

In [15]:
df_in_order = df_in_order[new_cols]
df_in_order

,name,median Y1 $i$-band coadded depth in effective survey area,median Y3 $i$-band coadded depth in effective survey area,median Y6 $i$-band coadded depth in effective survey area,median Y10 $i$-band coadded depth in effective survey area,std in Y1 $i$-band coadded depth in effective survey area,std in Y3 $i$-band coadded depth in effective survey area,std in Y6 $i$-band coadded depth in effective survey area,std in Y10 $i$-band coadded depth in effective survey area,Y1 effective survey area,Y3 effective survey area,Y6 effective survey area,Y10 effective survey area
0,alt_base_v1.6_10yrs.db,25.09,25.71,26.05,26.26,0.16,0.14,0.13,0.11,15325.03,15393.01,15328.86,15346.90
1,barebones_nexp2_v1.6_10yrs.db,25.20,25.81,26.14,26.40,0.19,0.15,0.14,0.13,15163.15,15326.81,15288.94,15282.23
2,barebones_v1.6_10yrs.db,25.21,25.86,26.19,26.46,0.19,0.15,0.14,0.13,15198.09,15382.68,15355.87,15339.35
3,baseline_nexp1_v1.6_10yrs.db,25.11,25.73,26.07,26.34,0.18,0.13,0.13,0.12,15165.04,15390.34,15353.99,15334.00
4,baseline_nexp2_scaleddown_v1.6_10yrs.db,25.06,25.69,26.03,26.30,0.17,0.13,0.13,0.12,14863.26,15257.10,15177.94,15178.68
5,baseline_nexp2_v1.6_10yrs.db,25.03,25.67,26.01,26.27,0.17,0.13,0.12,0.12,14836.77,15246.82,15188.64,15159.64
6,combo_dust_nexp2_v1.6_10yrs.db,24.97,25.56,25.91,26.18,0.18,0.13,0.13,0.12,15765.87,17556.45,17194.56,17087.49
7,combo_dust_v1.6_10yrs.db,25.03,25.63,25.98,26.24,0.19,0.14,0.14,0.13,17161.14,18024.20,17771.62,17693.36
8,ddf_heavy_nexp2_v1.6_10yrs.db,24.98,25.56,25.89,26.16,0.14,0.10,0.10,0.09,14262.06,15029.02,14827.70,14765.80
9,ddf_heavy_v1.6_10yrs.db,25.06,25.64,25.97,26.23,0.16,0.12,0.11,0.10,14944.15,15235.75,15129.58,15089.35


In [16]:
# save the metrics
fname = 'static_metrics.csv'
df_in_order.to_csv('%s/%s' % (outdir, fname), index=False)

#### now read and save lss-fom and ngal

In [17]:
# clean up the give_list dataframe
for key in df_in_order.keys():
    if key.__contains__('Y'):
        df_in_order = df_in_order.drop(key, axis=1)
        
df_in_order

,name
0,alt_base_v1.6_10yrs.db
1,barebones_nexp2_v1.6_10yrs.db
2,barebones_v1.6_10yrs.db
3,baseline_nexp1_v1.6_10yrs.db
4,baseline_nexp2_scaleddown_v1.6_10yrs.db
5,baseline_nexp2_v1.6_10yrs.db
6,combo_dust_nexp2_v1.6_10yrs.db
7,combo_dust_v1.6_10yrs.db
8,ddf_heavy_nexp2_v1.6_10yrs.db
9,ddf_heavy_v1.6_10yrs.db


#### read in the lss fom values

here we have to massage things a thing since we have two dfferent baselines, nexp1 and nexp2.

In [18]:
data_dir_base = '/global/cscratch1/sd/awan/lsst_output/post_wp_output_v1.6_-0.1cuts/lss_fom/'

In [19]:
# nexp1
data_dir1 = '%s/os_bias_overplots_nexp1/' % data_dir_base
[f for f in os.listdir(data_dir1) if f.__contains__('fomdata')]

['2021-03-23_fomdata_Y10_i<25.2_snfom_egfootprint_32cadences_0.66<z<1.0_th-r<25.60_100<ell<300_logtcuts.csv',
 '2021-03-23_fomdata_Y1_i<23.95_snfom_egfootprint_32cadences_0.66<z<1.0_th-r<24.35_100<ell<300_logtcuts.csv']

In [20]:
# nexp2
data_dir2 = '%s/os_bias_overplots_nexp2/' % data_dir_base
[f for f in os.listdir(data_dir2) if f.__contains__('fomdata')]

['2021-03-23_fomdata_Y10_i<25.2_snfom_egfootprint_32cadences_0.66<z<1.0_th-r<25.60_100<ell<300_logtcuts.csv',
 '2021-03-23_fomdata_Y1_i<23.95_snfom_egfootprint_32cadences_0.66<z<1.0_th-r<24.35_100<ell<300_logtcuts.csv']

In [21]:
fname = [f for f in os.listdir(data_dir1) if f.__contains__('fomdata') if f.__contains__('_Y1_')][0]
print(fname)
data_file = '%s/%s' % (data_dir1, fname)
lss_fom1 = pd.read_csv(data_file)

fname = [f for f in os.listdir(data_dir2) if f.__contains__('fomdata') if f.__contains__('_Y1_')][0]
data_file = '%s/%s' % (data_dir2, fname)
lss_fom2 = pd.read_csv(data_file)

lss_fom_combined = np.zeros_like(lss_fom1['FoM']) # initate the array

# find the indices for exp2 first since they actually have a tag in the name
for i, db in enumerate( df_in_order['name'] ):
    if db.__contains__('nexp2'):
        ind_temp = np.where( lss_fom2['dbname'].values == db.split('.db')[0] )[0]
    else:
        ind_temp = np.where( lss_fom1['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
    
    if db.__contains__('nexp2'):
        lss_fom_combined[i] = lss_fom2['FoM'].values[ind_temp]
    else:
        lss_fom_combined[i] = lss_fom1['FoM'].values[ind_temp]

df_in_order['Y1 lss fom'] = lss_fom_combined

2021-03-23_fomdata_Y1_i<23.95_snfom_egfootprint_32cadences_0.66<z<1.0_th-r<24.35_100<ell<300_logtcuts.csv


In [22]:
fname = [f for f in os.listdir(data_dir1) if f.__contains__('fomdata') if f.__contains__('_Y10_')][0]
print(fname)
data_file = '%s/%s' % (data_dir1, fname)
lss_fom1 = pd.read_csv(data_file)

fname = [f for f in os.listdir(data_dir2) if f.__contains__('fomdata') if f.__contains__('_Y10_')][0]
data_file = '%s/%s' % (data_dir2, fname)
lss_fom2 = pd.read_csv(data_file)

lss_fom_combined = np.zeros_like(lss_fom1['FoM']) # initate the array

# find the indices for exp2 first since they actually have a tag in the name
for i, db in enumerate( df_in_order['name'] ):
    if db.__contains__('nexp2'):
        ind_temp = np.where( lss_fom2['dbname'].values == db.split('.db')[0] )[0]
    else:
        ind_temp = np.where( lss_fom1['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
    
    if db.__contains__('nexp2'):
        lss_fom_combined[i] = lss_fom2['FoM'].values[ind_temp]
    else:
        lss_fom_combined[i] = lss_fom1['FoM'].values[ind_temp]

df_in_order['Y10 lss fom'] = lss_fom_combined

2021-03-23_fomdata_Y10_i<25.2_snfom_egfootprint_32cadences_0.66<z<1.0_th-r<25.60_100<ell<300_logtcuts.csv


In [23]:
ind = []
for db in df_in_order['name']:
    ind_temp = np.where( lss_fom1['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
        
    if ind_temp in ind:
        print('%s ind already in. %s' % (ind_temp, db))
    ind +=  list( ind_temp )
    
ind = np.array(ind)
print(ind)

lss_fom1['FoM'].values[ind]

[31 30 24 19 25 13 23 10  5 20  3 12 22 14 27 17  1 29  6 18 15  4  2 21
 16 28  8 26 11  0  9  7]


array([0.97756668, 0.9757458 , 0.9800202 , 0.97956298, 0.97435456,
       0.97366618, 1.0308394 , 1.04941451, 0.96269669, 0.97339763,
       0.97249814, 0.97845315, 0.97882379, 0.9781402 , 0.97911147,
       0.97809169, 0.97915921, 0.98658024, 1.03498554, 1.05235882,
       0.97836599, 0.97860757, 0.97882938, 0.97939851, 0.97913463,
       0.97884362, 0.97972506, 0.97951871, 0.97911811, 0.97862309,
       0.96053362, 0.97397463])

lets confirm things are working by visually check the Y10 additions

In [24]:
ind = []
for db in df_in_order['name']:
    ind_temp = np.where( lss_fom2['dbname'].values == db.split('.db')[0] )[0]

    if len(ind_temp) == 0:
        print('%s not found in data dict' % db)
        #ind_temp = np.array([ 0 ])
        
    if ind_temp in ind:
        print('%s ind already in. %s' % (ind_temp, db))
    ind +=  list( ind_temp )
    
ind = np.array(ind)
print(ind)

lss_fom2['FoM'].values[ind]

[31 30 24 19 25 13 23 10  5 20  3 12 22 14 27 17  1 29  6 18 15  4  2 21
 16 28  8 26 11  0  9  7]


array([0.98317249, 0.98134118, 0.98564009, 0.98518025, 0.97994196,
       0.97924963, 1.03675071, 1.05543233, 0.96821724, 0.97897954,
       0.9780749 , 0.98406405, 0.98443682, 0.98374931, 0.98472615,
       0.98370052, 0.98477417, 0.99223774, 1.04092063, 1.05839354,
       0.98397639, 0.98421936, 0.98444244, 0.98501483, 0.98474944,
       0.98445676, 0.98534325, 0.98513573, 0.98473283, 0.98423496,
       0.96604176, 0.97955986])

In [25]:
df_in_order

,name,Y1 lss fom,Y10 lss fom
0,alt_base_v1.6_10yrs.db,0.930114,0.977567
1,barebones_nexp2_v1.6_10yrs.db,0.898994,0.981341
2,barebones_v1.6_10yrs.db,0.924743,0.980020
3,baseline_nexp1_v1.6_10yrs.db,0.935198,0.979563
4,baseline_nexp2_scaleddown_v1.6_10yrs.db,0.921073,0.979942
5,baseline_nexp2_v1.6_10yrs.db,0.917659,0.979250
6,combo_dust_nexp2_v1.6_10yrs.db,0.921372,1.036751
7,combo_dust_v1.6_10yrs.db,0.957700,1.049415
8,ddf_heavy_nexp2_v1.6_10yrs.db,0.903540,0.968217
9,ddf_heavy_v1.6_10yrs.db,0.922243,0.973398


things appear to be working fine

#### now read in the ngal values

In [26]:
ngal_data_dir = data_dir_base

ngal_data = {}
for yr in [1, 10]:
    ngal_data[yr] =  np.zeros(len(df_in_order.name))
    folders = [f for f in os.listdir( ngal_data_dir ) if f.__contains__('_Y%s_' % yr) and f.__contains__('i<') ]
    for i, folder in enumerate( folders ):
        out = pd.read_csv('%s/%s/ngal-final_total/ngal_NoDither.txt' % (ngal_data_dir, folder), header=None, delimiter=' ') 
        dbname, ngal = out[0].values[0], out[1].values[0]
        #print(dbname)
        ind = np.where( df_in_order['name'].values == '%s.db' % dbname)[0]
        ngal_data[yr][ind] = ngal
    print('read in %s folders for Y%s' % (i+1, yr) )

read in 32 folders for Y1
read in 32 folders for Y10


In [27]:
for yr in ngal_data:
    df_in_order['Y%s Ngal (0.66<z<1.0)' % yr] = ngal_data[yr]

In [28]:
df_in_order

,name,Y1 lss fom,Y10 lss fom,Y1 Ngal (0.66<z<1.0),Y10 Ngal (0.66<z<1.0)
0,alt_base_v1.6_10yrs.db,0.930114,0.977567,448000000.0,9.560000e+08
1,barebones_nexp2_v1.6_10yrs.db,0.898994,0.981341,444000000.0,9.590000e+08
2,barebones_v1.6_10yrs.db,0.924743,0.980020,445000000.0,9.640000e+08
3,baseline_nexp1_v1.6_10yrs.db,0.935198,0.979563,443000000.0,9.610000e+08
4,baseline_nexp2_scaleddown_v1.6_10yrs.db,0.921073,0.979942,434000000.0,9.500000e+08
5,baseline_nexp2_v1.6_10yrs.db,0.917659,0.979250,433000000.0,9.480000e+08
6,combo_dust_nexp2_v1.6_10yrs.db,0.921372,1.036751,459000000.0,1.060000e+09
7,combo_dust_v1.6_10yrs.db,0.957700,1.049415,500000000.0,1.100000e+09
8,ddf_heavy_nexp2_v1.6_10yrs.db,0.903540,0.968217,415000000.0,9.190000e+08
9,ddf_heavy_v1.6_10yrs.db,0.922243,0.973398,436000000.0,9.420000e+08


In [29]:
fname = 'lss_metrics.csv'
df_in_order.to_csv('%s/%s' % (outdir, fname), index=False)